Creating a ResNet50 to model the Chest X-Ray Images (Pneumonia) dataset from Kaggle (from http://www.cell.com/cell/fulltext/S0092-8674(18)30154-5). This model is taken from the Residual_Networks_v2a Notebook from week 2 of Andrew Ng's Convolutional Neural Networks course on Coursera.

In [22]:
import numpy as np
import os
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Dropout, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras import applications
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.optimizers import SGD, Adam
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [16]:
#Files are separated into separate folders for train, test, and validation, and also split
#into separate folders for + and - pneumonia.

# Put all the image filepaths into lists

test_neg_filepaths = []
for dirname, _, filenames in os.walk("../input/chest-xray-pneumonia/chest_xray/test/NORMAL"):
    for filename in filenames:
        test_neg_filepaths.append(os.path.join(dirname, filename))
        
test_pos_filepaths = []
for dirname, _, filenames in os.walk("../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA"):
    for filename in filenames:
        test_pos_filepaths.append(os.path.join(dirname, filename))
        
train_neg_filepaths = []
for dirname, _, filenames in os.walk("../input/chest-xray-pneumonia/chest_xray/train/NORMAL"):
    for filename in filenames:
        train_neg_filepaths.append(os.path.join(dirname, filename))
        
train_pos_filepaths = []
for dirname, _, filenames in os.walk("../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA"):
    for filename in filenames:
        train_pos_filepaths.append(os.path.join(dirname, filename))
        
val_neg_filepaths = []
for dirname, _, filenames in os.walk("../input/chest-xray-pneumonia/chest_xray/val/NORMAL"):
    for filename in filenames:
        val_neg_filepaths.append(os.path.join(dirname, filename))
        
val_pos_filepaths = []
for dirname, _, filenames in os.walk("../input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA"):
    for filename in filenames:
        val_pos_filepaths.append(os.path.join(dirname, filename))
        
print("Number of training +: " + str(len(train_pos_filepaths)))
print("Number of training -: " + str(len(train_neg_filepaths)))
print("Number of validation +: " + str(len(val_pos_filepaths)))
print("Number of validation -: " + str(len(val_neg_filepaths)))
print("Number of test +: " + str(len(test_pos_filepaths)))
print("Number of test -: " + str(len(test_neg_filepaths)))
m_train_p=len(train_pos_filepaths)
m_train_n=len(train_neg_filepaths)
m_val_p=len(val_pos_filepaths)
m_val_n=len(val_neg_filepaths)
m_test_p=len(test_pos_filepaths)
m_test_n=len(test_neg_filepaths)

Number of training +: 3875
Number of training -: 1341
Number of validation +: 8
Number of validation -: 8
Number of test +: 390
Number of test -: 234


In [17]:
# Create the y data vectors, first column is no pneumonia, second column is neumonia.

y_train_pos = np.ones((3875, 1))
y_train_neg = np.zeros((1341, 1))
y_val_pos = np.ones((8, 1))
y_val_neg = np.zeros((8, 1))
y_test_pos = np.ones((390, 1))
y_test_neg = np.zeros((234, 1))

y_train = np.concatenate((y_train_pos, y_train_neg))
y_val = np.concatenate((y_val_pos, y_val_neg))
y_test = np.concatenate((y_test_pos, y_test_neg))

def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y

y_train_oh = convert_to_one_hot(y_train.astype(int), 2).T
y_val_oh = convert_to_one_hot(y_val.astype(int), 2).T
y_test_oh = convert_to_one_hot(y_test.astype(int), 2).T

In [18]:
# Load the image data into arrays.

target_size = 128
n_H_max = target_size
n_W_max = target_size
x_train_pos = np.zeros((3875, n_H_max, n_W_max, 3))
x_train_neg = np.zeros((1341, n_H_max, n_W_max, 3))
x_val_pos = np.zeros((8, n_H_max, n_W_max, 3))
x_val_neg = np.zeros((8, n_H_max, n_W_max, 3))
x_test_pos = np.zeros((390, n_H_max, n_W_max, 3))
x_test_neg = np.zeros((234, n_H_max, n_W_max, 3))


for path in train_pos_filepaths:
    img = image.load_img(path, target_size=(target_size, target_size))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = x/255.0
    x_train_pos[0] = x
    
for path in train_neg_filepaths:
    img = image.load_img(path, target_size=(target_size, target_size))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = x/255.0
    x_train_neg[0] = x   
for path in val_pos_filepaths:
    img = image.load_img(path, target_size=(target_size, target_size))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = x/255.0
    x_val_pos[0] = x
    
for path in val_neg_filepaths:
    img = image.load_img(path, target_size=(target_size, target_size))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = x/255.0
    x_val_neg[0] = x
    
for path in test_pos_filepaths:
    img = image.load_img(path, target_size=(target_size, target_size))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = x/255.0
    x_test_pos[0] = x
    
for path in train_neg_filepaths:
    img = image.load_img(path, target_size=(target_size, target_size))
    x = image.img_to_array(img)
    #x = np.expand_dims(x, axis=0)
    x = x/255.0
    x_test_neg[0] = x

x_train = np.concatenate((x_train_pos, x_train_neg), axis = 0)
x_test = np.concatenate((x_test_pos, x_test_neg), axis = 0)
x_val = np.concatenate((x_val_pos, x_val_neg), axis = 0)

print("x_train: " + str(x_train.shape))
print("y_train: " + str(y_train.shape))
print("x_val: " + str(x_val.shape))
print("y_val: " + str(y_val.shape))
print("x_test: " + str(x_test.shape))
print("y_test: " + str(x_test.shape))

x_train: (5216, 128, 128, 3)
y_train: (5216, 1)
x_val: (16, 128, 128, 3)
y_val: (16, 1)
x_test: (624, 128, 128, 3)
y_test: (624, 128, 128, 3)


In [19]:
input_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (n_H_max,n_W_max,3))

In [24]:
x = input_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(2, activation= 'softmax')(x)
model = Model(inputs = input_model.input, outputs = predictions)

NameError: name 'base_model' is not defined

In [ ]:
adam = Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train_oh, epochs = 1, batch_size = 32)

In [ ]:
val_preds = model.evaluate(x_val, y_val_oh)
print ("Loss = " + str(val_preds[0]))
print ("Val Accuracy = " + str(val_preds[1]))

In [ ]:
test_preds = model.evaluate(x_test, y_test_oh)
print ("Loss = " + str(test_preds[0]))
print ("Test Accuracy = " + str(test_preds[1]))